## Chapter 5   
### 순전파와 역전파   
문제를 계산그래프로 풀 때    
<li>왼쪽에서 오른쪽 : 순전파</li>   
<li>오른쪽에서 왼쪽 : 역전파</li>   
계산그래프의 특징으로는 <b>국소적 계산</b>이 있다.   
각각의 계산은 앞의 계산과는 상관없이 본인의 계산만 하면 된다. 따라서 역전파를 했을 때 미분을 효율적으로 계산할 수 있는 장점이 있다.   

### 연쇄법칙   
<i>합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.</i>   
$$ {\delta z \over \delta x} = {\delta z \over \delta t}{\delta t \over \delta x} $$   
이를 통해 계산 그래프의 역전파는 국소적 미분값들의 곱을 통해 최종적으로 미분값을 나타낼 수 있다.   

### 덧셈과 곱셈 노드의 역전파   
<b> 덧셈노드</b>
$ {\delta L \over \delta x }  $   
<br>
<b> 곱셈노드</b>
$ {\delta L \over \delta x }  {\delta x \over \delta y} $

덧셈노드의 경우 1을 곱하는 것이기 때문에 입력 신호 값이 필요 없지만, 곱셈노드의 경우 순방향 입력 신호 값이 필요하기 때문에 순전파의 입력 신호를 저장해둔다.   

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y

        return out
    
    def backward(self, dout): #역전파
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer= MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [2]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [4]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout  * 1
        dy = dout * 1
        return dx, dy

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

print(price)
print(dapple, dapple_num, dorange, dorange_num, dtax)


715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


계산 그래프에서 계층의 구현은 쉽게 구현할 수 있으며, 이를 사용해 복잡한 미분도 효율적으로 계산할 수 있다. 